# Q4_Answer

运行 TwoDimDiffusion.zip 中的程序，体会、理解和掌握有限差分和有限体积法的求解 2D 热传导方程的基本步骤。掌握输出给定点温度值的方法

## 第一步：初始化相关参数

In [1]:
import numpy as np

col_len = 30  # 列数
row_len = 50  # 行数
node_num = col_len * row_len  # 网格数量
tlist_new = np.empty([row_len, col_len])  # 记录新温度
tlist_old = np.array([[20.0] * col_len for row in range(row_len)])  # 记录旧温度
t_left = 100.0  # 左边界侧温度

max_loops = 10000  # 迭代次数
step_before_1w = 500  # 迭代次数为 1w 前（含）步长
step_after_1w = 10000  # 迭代次数为 1w 后步长
loop_count = 0  # 迭代保存计数器

In [2]:
delta_x = 0.5
delta_t = 1
alpha = 0.005
alpha_dimless = alpha / (delta_x * delta_x / delta_t)

## 第二步：定义有限差分算法（向量法）

分别使用 numba 库和 numpy 库提速，整体提速 30 倍

In [3]:
from numba import jit  # 使用 numba 库提高计算性能，10w loop 也能 3s 出结果

@jit(nopython=True)
def thermal_conductivity(tlist_new, tlist_old):  # 使用向量计算提高速度
    # 去 左 右 下 边界区域
    for row in range(row_len-1):  # 去下边界
        tlist_new[row] = tlist_old[row-1] + \
            tlist_old[row+1] - 2 * tlist_old[row]

    for col in range(1, col_len-1):  # 去 左右 边界
        tlist_new[:, col] = tlist_old[:, col-1] + \
            tlist_old[:, col+1] - 2 * tlist_old[:, col]

    tlist_new[-1] = tlist_old[-2] + tlist_old[0] - 2 * tlist_old[-1]  # 下边界

    tlist_new = alpha_dimless * tlist_new + tlist_old

    tlist_new[:, 0] = t_left  # 左边界
    tlist_new[:, -1] = tlist_new[:, -2]  # 右边界: partial T / partial x = 0

    return tlist_new

## 第三步：启动循环

In [4]:
thermal_data = []
append = thermal_data.append
for loops in range(max_loops+1):
    tlist_old = thermal_conductivity(tlist_new, tlist_old)
    if loops % step_before_1w == 0:
        if loops <= 10000 or loops % step_after_1w == 0:
            append(tlist_old)

## 第五步：绘图

In [5]:
# 导入绘图库
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [6]:
data = [go.Heatmap(visible=False, zmin=20, zmax=100, z=t,
                   hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> ") for t in thermal_data]

data1 = [go.Surface(visible=False, z=t,
                    hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> ") for t in thermal_data]

data1[0]['visible'] = data[0]['visible'] = True

In [7]:
steps = []
data_len = len(data)
for i in range(data_len):
    step = dict(
        method='restyle',
        args=['visible', [False] * data_len],
        label=i*step_before_1w
    )
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "loop: "},
    steps=steps
)]

layout = dict(sliders=sliders, width = 800, height = 800)

fig = dict(data=data, layout=layout)
fig1 = dict(data=data1, layout=layout)
plotly.offline.iplot(fig)
plotly.offline.iplot(fig1)